<a href="https://colab.research.google.com/github/diogocezar/phd-machine-learning-lab3/blob/master/machine_learning_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aprendizagem de Máquina - LAB 3

A ideia deste notebook é criar um experimento compartilhado entre os alunos, para que a atividade de laboratório 3 da disciplina de aprenziagem de máquina seja executada.

## Enunciado do Problema

Para esse laboratório considere a base de dados de meses do ano (12 classes) apresentado nas práticas de Deep Learning.

1. Implemente funções para aumentar o número de amostras do conjunto de TREINAMENTO (Data Augmentation);

2. Implemente duas redes neurais convolucionais: (a) LeNet 5 [Yann LeCun (1998)]; (b) CNN de sua escolha;

3. Escreva um breve relatório que:
  - Descreva as CNNs utilizadas e as funções de Data Augmentation;
  - Compare o desempenho dessas redes variando os diferentes parâmetros apresentados em aula. Realize treinamentos com e sem Data Augmentation. Analise os resultados obtidos nos diferentes experimentos apresentando suas conclusões (apresente gráficos e matriz de confusão);
  - Repita os experimentos e análises utilizando 2 redes pré-treinadas na ImageNet (Transfer Learning/Fine-Tuning). Utilize essas redes para gerar vetores de características, realizando a classificação em outro classificador (Ex: SVM).

O relatório reportando seus experimentos deve entregue em formato PDF.

IMPORTANTE:

Somente arquivos em PDF serão corrigidos. 
Respeite o prazo de entrega. O moodle não aceitará envios após a Data Limite.

## Links

https://github.com/TaavishThaman/LeNet-5-with-Keras/blob/master/lenet_5.py

# Solução do Adriano

```
from keras.datasets import mnist
from keras.utils import np_utils
# Load dataset as train and test sets
from Lab03.functions_load import load_dataset
from keras.models import Sequential
from keras import models, layers
import keras
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

#https://engmrk.com/lenet-5-a-classic-cnn-architecture/

drive_path = '../Lab03/meses/'

## Train and Test files
train_file = drive_path + 'train.txt'
test_file = drive_path + 'test.txt'

## Input Image Dimension
img_rows, img_cols = 32, 32

num_classes = 12

n_epochs = 64

(x_train, y_train), (x_test, y_test) = load_dataset(train_file, test_file, resize=True, convert=True, size=(img_rows, img_cols))

## Save for the confusion matrix
label = []
for i in range(len(x_test)):
	label.append(y_test[i][0])

# Set numeric type to float32 from uint8
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize value to [0, 1]
x_train /= 255
x_test /= 255

## Convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

#Instantiate an empty model
model = Sequential()

# C1 Convolutional Layer
model.add(layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=(img_rows,img_cols,3), padding="same"))

# S2 Pooling Layer
model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))

# C3 Convolutional Layer
model.add(layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))

# S4 Pooling Layer
model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

# C5 Fully Connected Convolutional Layer
model.add(layers.Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))
#Flatten the CNN output so that we can connect it with fully connected layers
model.add(layers.Flatten())

# FC6 Fully Connected Layer
model.add(layers.Dense(84, activation='tanh'))

#Output Layer with softmax activation
model.add(layers.Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=["accuracy"])

hist = model.fit(x=x_train, y=y_train, epochs=n_epochs, batch_size=128, validation_data=(x_test, y_test), verbose=1)

test_score = model.evaluate(x_test, y_test)
print("Test loss {:.4f}, accuracy {:.2f}%".format(test_score[0], test_score[1] * 100))

## Print CNN layers
print('Network structure ----------------------------------')

# for i, layer in enumerate(model.layers):
# 	print(i,layer.name)
# 	if hasattr(layer, 'output_shape'):
# 		print(layer.output_shape)

model.summary()

print('----------------------------------------------------')

score = model.evaluate(x_test, y_test, verbose=0)
print('\n----------------------------------------------------\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('\n----------------------------------------------------\n')

## Classes predicted
# print (model.predict_classes(x_test))

## Classes probability
# print (model.predict_proba(x_test))

"""#Matriz de confusão"""

pred = []
y_pred = model.predict_classes(x_test)
# y_pred = y_prob.argmax(axis=-1)
for i in range(len(x_test)):
    pred.append(y_pred[i])
print(confusion_matrix(label, pred))

"""#Plotando gráficos"""

acc = hist.history['accuracy']  # history['acc'] / history['accuracy']
val_acc = hist.history['val_accuracy']  # history['val_acc'] / history['val_accuracy']
loss = hist.history['loss']
val_loss = hist.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Acurácia do treinamento')
plt.plot(epochs, val_acc, 'r', label='Acurácia da validação')
plt.title('Acurácia do treinamento e validação')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Perda do treinamento')
plt.plot(epochs, val_loss, 'r', label='Perda da validação')
plt.title('Perda do treinamento e validação')
plt.legend()

plt.show()
```

# Passo 1 - Verificando a GPU

Neste ponto se importa o tensorflow e executa-se uma verificação para analisar se a máquina que executará os scripts utiliza o processamento em GPU.

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


Caso a saída seja: `Found...` então, pode-se executar a próxia linha, que habilita a execução.

In [ ]:
!nvidia-smi

Fri Aug 28 18:14:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    28W /  70W |    227MiB / 15079MiB |      1%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Passo 2 - Obtendo os Dados

Neste ponto, utiliza-se o repositório do GitHub listado abaixo para obter o .zip com as informações a serem utilizadas no experimento.

Os comandos na sequência removem as pastas desnecessárias deixando apenas os arquivos importantes para a execução dos próximos passos.

In [ ]:

!git clone https://github.com/diogocezar/phd-machine-learning-lab3
!mv phd-machine-learning-lab3/months.zip .
!unzip months.zip
!rm -Rf months.zip sample_data __MACOSX phd-machine-learning-lab3

Cloning into 'phd-machine-learning-lab3'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 6 (delta 0), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.
Archive:  months.zip
  inflating: train.txt               
   creating: __MACOSX/
  inflating: __MACOSX/._train.txt    
  inflating: test.txt                
  inflating: __MACOSX/._test.txt     
   creating: data/
  inflating: data/AD0498.jpg         
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._AD0498.jpg  
  inflating: data/td0143.jpg         
  inflating: __MACOSX/data/._td0143.jpg  
  inflating: data/AD0329.jpg         
  inflating: __MACOSX/data/._AD0329.jpg  
  inflating: data/AD1037.jpg         
  inflating: __MACOSX/data/._AD1037.jpg  
  inflating: data/AD1023.jpg         
  inflating: __MACOSX/data/._AD1023.jpg  
  inflating: data/AD0315.jpg         
  inflating: __MACOSX/data/._AD0315.jpg  
  infl

# Passo 3 - Aumentando o Número de Amostras

## Solução do Adriano

```
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
import glob
import os
from tempfile import mkstemp
from shutil import move, copymode
from os import fdopen, remove

#https://machinelearningmastery.com/how-to-configure-image-data-augmentation-when-training-deep-learning-neural-networks/

drive_path = '../Lab03/meses/'
aug_path = drive_path + "data-aug"
train_file_aug = drive_path + 'train-aug.txt'


def replace(file_path, pattern):
    #Create temp file
    fh, abs_path = mkstemp()
    with fdopen(fh,'w') as new_file:
        with open(file_path) as old_file:
            for line in old_file:
                print(pattern in line)
                if pattern in line:
                    continue
                new_file.write(line)
    #Copy the file permissions from the old file to the new file
    copymode(file_path, abs_path)
    #Remove original file
    remove(file_path)
    #Move new file
    move(abs_path, file_path)

def save_to_aug(label, subdir):
    list_of_files = glob.glob(aug_path + "/" + subdir + "/*")  # * means all if need specific format then *.csv
    if len(list_of_files) > 0:
        latest_file = max(list_of_files, key=os.path.getctime)
        replace(train_file_aug, latest_file.replace(aug_path + "/", ""))
        arq_aug = open(train_file_aug, "a+")
        arq_aug.write(latest_file.replace(aug_path + "/", "") + " " + label + "\n")
        arq_aug.close()


def flip_rotation_brightness_zoom(path, zoom=[0.5, 1.0], brightness=[0.2, 1.0], rotation=90, flip_horizontal=False,
                                  flip_vertical=False, subdir="all"):
    path, label = path.split(' ')
    path = drive_path + 'data/' + path
    img = load_img(path)
    # convert to numpy array
    data = img_to_array(img)
    # expand dimension to one sample
    samples = expand_dims(data, 0)
    # create image data augmentation generator
    datagen = ImageDataGenerator(zoom_range=zoom, brightness_range=brightness, rotation_range=rotation,
                                 horizontal_flip=flip_horizontal,vertical_flip=flip_vertical)
    # prepare iterator
    it = datagen.flow(samples, save_to_dir=aug_path + "/" + subdir + "/", batch_size=1)
    save_to_aug(label, subdir)
    # generate samples and plot
    for i in range(1):
        # define subplot
        pyplot.subplot(330 + 1 + i)
        # generate batch of images
        batch = it.next()
        # convert to unsigned integers for viewing
        image = batch[0].astype('uint8')
        # plot raw pixel data
        pyplot.imshow(image)
    # show the figure
    # pyplot.show()


def random_zoom(path, zoom=[0.5, 1.0], subdir="zoom"):
    path, label = path.split(' ')
    path = drive_path + 'data/' + path
    img = load_img(path)
    # convert to numpy array
    data = img_to_array(img)
    # expand dimension to one sample
    samples = expand_dims(data, 0)
    # create image data augmentation generator
    datagen = ImageDataGenerator(zoom_range=[0.5, 1.0])
    # prepare iterator
    it = datagen.flow(samples, save_to_dir=aug_path + "/" + subdir + "/", batch_size=1)
    save_to_aug(label, subdir)
    # generate samples and plot
    for i in range(1):
        # define subplot
        pyplot.subplot(330 + 1 + i)
        # generate batch of images
        batch = it.next()
        # convert to unsigned integers for viewing
        image = batch[0].astype('uint8')
        # plot raw pixel data
        pyplot.imshow(image)
    # show the figure
    #pyplot.show()


def random_brightness(path, brightness=[0.2, 1.0], subdir="brightness"):
    path, label = path.split(' ')
    path = drive_path + 'data/' + path
    # load the image
    img = load_img(path)
    # convert to numpy array
    data = img_to_array(img)
    # expand dimension to one sample
    samples = expand_dims(data, 0)
    # create image data augmentation generator
    datagen = ImageDataGenerator(brightness_range=brightness)
    # prepare iterator
    it = datagen.flow(samples, save_to_dir=aug_path + "/" + subdir + "/", batch_size=1)
    save_to_aug(label, subdir)
    # generate samples and plot
    for i in range(1):
        # define subplot
        pyplot.subplot(330 + 1 + i)
        # generate batch of images
        batch = it.next()
        # convert to unsigned integers for viewing
        image = batch[0].astype('uint8')
        # plot raw pixel data
        pyplot.imshow(image)
    # show the figure
    #pyplot.show()


def random_rotation(path, rotation=90, subdir="rotation"):
    path, label = path.split(' ')
    path = drive_path + 'data/' + path
    img = load_img(path)
    # convert to numpy array
    data = img_to_array(img)
    # expand dimension to one sample
    samples = expand_dims(data, 0)
    # create image data augmentation generator
    datagen = ImageDataGenerator(rotation_range=rotation)
    # prepare iterator
    it = datagen.flow(samples, save_to_dir=aug_path + "/" + subdir + "/", batch_size=1)
    save_to_aug(label, subdir)
    # generate samples and plot
    for i in range(1):
        # define subplot
        pyplot.subplot(330 + 1 + i)
        # generate batch of images
        batch = it.next()
        # convert to unsigned integers for viewing
        image = batch[0].astype('uint8')
        # plot raw pixel data
        pyplot.imshow(image)
    # show the figure
    #pyplot.show()


def horizontal_vertical_flip(path, flip_horizontal=False, flip_vertical=False, subdir="flip"):
    path, label = path.split(' ')
    path = drive_path + 'data/' + path
    # load the image
    img = load_img(path)
    # convert to numpy array
    data = img_to_array(img)
    # expand dimension to one sample
    samples = expand_dims(data, 0)
    # create image data augmentation generator
    datagen = ImageDataGenerator(horizontal_flip=flip_horizontal,vertical_flip=flip_vertical)
    # prepare iterator
    it = datagen.flow(samples, save_to_dir=aug_path + "/" + subdir + "/", batch_size=1)
    save_to_aug(label, subdir)
    # generate samples and plot
    for i in range(1):
        # define subplot
        pyplot.subplot(330 + 1 + i)
        # generate batch of images
        batch = it.next()
        # convert to unsigned integers for viewing
        image = batch[0].astype('uint8')
        # plot raw pixel data
        pyplot.imshow(image)
    # show the figure
    #pyplot.show()


def horizontal_vertical_shift(path, size=0.5, bool_width=True, subdir="shift"):
    path, label = path.split(' ')
    path = drive_path + 'data/' + path
    # load the image
    img = load_img(path)
    # convert to numpy array
    data = img_to_array(img)
    # expand dimension to one sample
    samples = expand_dims(data, 0)
    # create image data augmentation generator
    if bool_width:
        datagen = ImageDataGenerator(width_shift_range=size)
    else:
        datagen = ImageDataGenerator(height_shift_range=size)
    # prepare iterator
    it = datagen.flow(samples, save_to_dir=aug_path + "/" + subdir + "/", batch_size=1)
    save_to_aug(label, subdir)
    # generate samples and plot
    for i in range(1):
        # define subplot
        pyplot.subplot(330 + 1 + i)
        # generate batch of images
        batch = it.next()
        # convert to unsigned integers for viewing
        image = batch[0].astype('uint8')
        # plot raw pixel data
        pyplot.imshow(image)
    # show the figure
    #pyplot.show()


# Train and Test files
train_file = drive_path + 'train.txt'

arq = open(train_file, 'r')
texto = arq.read()
train_paths = texto.split('\n')

train_paths.remove('')  # Remove empty lines
train_paths.sort()

for image_path in train_paths:
    horizontal_vertical_shift(image_path, bool_width=True)
    horizontal_vertical_shift(image_path, bool_width=False)
    horizontal_vertical_flip(image_path, flip_horizontal=True)
    horizontal_vertical_flip(image_path, flip_vertical=True)
    random_rotation(image_path)
    random_rotation(image_path, rotation=45)
    random_brightness(image_path)
    random_brightness(image_path, brightness=[0, 0.2])
    random_zoom(image_path)
    random_zoom(image_path, zoom=[0, 0.5])
    flip_rotation_brightness_zoom(image_path)
    flip_rotation_brightness_zoom(image_path, zoom=[0.1, 0.5], brightness=[0.1, 0.5], flip_horizontal=True)
    flip_rotation_brightness_zoom(image_path, zoom=[0.1, 0.5], brightness=[0.1, 0.5], flip_vertical=True)
    flip_rotation_brightness_zoom(image_path, zoom=[0.1, 0.5], brightness=[0.1, 0.5], rotation=180)
    flip_rotation_brightness_zoom(image_path, zoom=[0.1, 0.5], brightness=[0.1, 0.5], rotation=180, flip_vertical=True)
    flip_rotation_brightness_zoom(image_path, zoom=[0.1, 0.5], brightness=[0.1, 0.5], rotation=180, flip_horizontal=True)
    flip_rotation_brightness_zoom(image_path, zoom=[0, 0.8], brightness=[0, 0.8], rotation=45,
                                  flip_vertical=True)
    flip_rotation_brightness_zoom(image_path, zoom=[0, 0.8], brightness=[0, 0.8], rotation=45,
                                  flip_vertical=True)
    flip_rotation_brightness_zoom(image_path, zoom=[0.1, 0.2], brightness=[0.1, 0.2], rotation=180)
    flip_rotation_brightness_zoom(image_path, zoom=[0.1, 0.2], brightness=[0.1, 0.2], rotation=45)
    flip_rotation_brightness_zoom(image_path, zoom=[0.9, 1], brightness=[0.9, 1], rotation=180)
    flip_rotation_brightness_zoom(image_path, zoom=[0.9, 1], brightness=[0.9, 1], rotation=45)
  ```

# Passo 3 - Implementando a Rede Convolucional LeNet 5

In [ ]:
# Imports

import numpy as np
import cv2
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

from PIL import Image

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras import backend as K

from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.models import Model
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing import image

In [ ]:
# Definitions

## Classes
num_classes = 12

## Batch Size
batch_size = 64

## Epochs
n_epochs = 64

## Train and Test files
train_file = './train.txt'
test_file = './test.txt'

## Input Image Dimension
img_rows, img_cols = 32, 32

In [ ]:
## Resize

def resize_data(data, size, convert):

	if convert:
		data_upscaled = np.zeros((data.shape[0], size[0], size[1], 3))
	else:
		data_upscaled = np.zeros((data.shape[0], size[0], size[1]))
	for i, img in enumerate(data):
		large_img = cv2.resize(img, dsize=(size[1], size[0]), interpolation=cv2.INTER_CUBIC)
		data_upscaled[i] = large_img
	return data_upscaled

In [ ]:
## Load Images

def load_images(image_paths, convert=False):

	x = []
	y = []
	for image_path in image_paths:

		path, label = image_path.split(' ')
		
		## Image path
		path= './data/' + path
		# print (path)

		if convert:
			image_pil = Image.open(path).convert('RGB') 
		else:
			image_pil = Image.open(path).convert('L')

		img = np.array(image_pil, dtype=np.uint8)

		x.append(img)
		y.append([int(label)])

	x = np.array(x)
	y = np.array(y)

	if np.min(y) != 0: 
		y = y-1

	return x, y

In [ ]:
def load_dataset(train_file, test_file, resize, convert=False, size=(224,224)):

	arq = open(train_file, 'r')
	texto = arq.read()
	train_paths = texto.split('\n')
	
	print ('Size:', size)

	train_paths.remove('') # Remove empty lines
	train_paths.sort()

	print ("Loading training set...")
	x_train, y_train = load_images(train_paths, convert)
 
	arq = open(test_file, 'r')
	texto = arq.read()
	test_paths = texto.split('\n')

	test_paths.remove('') # Remove empty lines
	test_paths.sort()
 
	print ("Loading testing set...")
	x_test, y_test = load_images(test_paths, convert)

	if resize:
		print ("Resizing images...")
		x_train = resize_data(x_train, size, convert)
		x_test = resize_data(x_test, size, convert)

	if not convert:
		x_train = x_train.reshape(x_train.shape[0], size[0], size[1], 1)
		x_test = x_test.reshape(x_test.shape[0], size[0], size[1], 1)

	print (np.shape(x_train))
 
	return (x_train, y_train), (x_test, y_test)

In [ ]:
## Loading Inital Data

input_shape = (img_rows, img_cols, 3)

(x_train, y_train), (x_test, y_test) = load_dataset(train_file, test_file, resize=True, convert=True, size=(img_rows, img_cols))

print ('\n',x_train.shape[0], 'train samples')
print ('\n',x_test.shape[0], 'test samples')

Size: (32, 32)
Loading training set...
Loading testing set...
Resizing images...
(1578, 32, 32, 3)

 1578 train samples

 401 test samples


In [ ]:
## Normalize images
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## Save for the confusion matrix
label = []
for i in range(len(x_test)):
	label.append(y_test[i][0])
 
## Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# LeNet 5 Model

#Instantiate an empty model
model = Sequential()

# C1 Convolutional Layer
model.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=(img_rows,img_cols,3), padding="same"))

# S2 Pooling Layer
model.add(AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))

# C3 Convolutional Layer
model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))

# S4 Pooling Layer
model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

# C5 Fully Connected Convolutional Layer
model.add(Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))
#Flatten the CNN output so that we can connect it with fully connected layers
model.add(Flatten())

# FC6 Fully Connected Layer
model.add(Dense(84, activation='tanh'))

#Output Layer with softmax activation
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=["accuracy"])

print ('Network structure ----------------------------------')
model.summary()
print ('----------------------------------------------------')

Network structure ----------------------------------
Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_201 (Conv2D)          (None, 32, 32, 6)         456       
_________________________________________________________________
average_pooling2d_30 (Averag (None, 31, 31, 6)         0         
_________________________________________________________________
conv2d_202 (Conv2D)          (None, 27, 27, 16)        2416      
_________________________________________________________________
average_pooling2d_31 (Averag (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_203 (Conv2D)          (None, 9, 9, 120)         48120     
_________________________________________________________________
flatten_6 (Flatten)          (None, 9720)              0         
_________________________________________________________________
d

In [ ]:
# Trainning

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='SGD', metrics=["accuracy"])

## Trains the model
history = model.fit(x=x_train, y=y_train, epochs=n_epochs, batch_size=128, validation_data=(x_test, y_test), verbose=1)

score = model.evaluate(x_test, y_test, verbose=0)
print ('\n----------------------------------------------------\n')
print ('Test loss:', score[0])
print ('Test accuracy:', score[1])
print ('\n----------------------------------------------------\n')

Epoch 1/64
13/13 [==============================] - 0s 20ms/step - loss: 0.5135 - accuracy: 0.8625 - val_loss: 0.8739 - val_accuracy: 0.7157
Epoch 2/64
13/13 [==============================] - 0s 11ms/step - loss: 0.5076 - accuracy: 0.8638 - val_loss: 0.8696 - val_accuracy: 0.7032
Epoch 3/64
13/13 [==============================] - 0s 10ms/step - loss: 0.5008 - accuracy: 0.8701 - val_loss: 0.8820 - val_accuracy: 0.7157
Epoch 4/64
13/13 [==============================] - 0s 9ms/step - loss: 0.4981 - accuracy: 0.8663 - val_loss: 0.8816 - val_accuracy: 0.6933
Epoch 5/64
13/13 [==============================] - 0s 9ms/step - loss: 0.4898 - accuracy: 0.8733 - val_loss: 0.8678 - val_accuracy: 0.7182
Epoch 6/64
13/13 [==============================] - 0s 8ms/step - loss: 0.4836 - accuracy: 0.8720 - val_loss: 0.8830 - val_accuracy: 0.7107
Epoch 7/64
13/13 [==============================] - 0s 8ms/step - loss: 0.4719 - accuracy: 0.8796 - val_loss: 0.8528 - val_accuracy: 0.7107
Epoch 8/64
13/13 

# Passo 4 - Implementando uma CNN (Sugestão SVM)

# Passo 5 - Implementando com Redes Pré-Treinadas ImageNet (Transfer Learning/Fine-Tuning).